In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from datetime import date
from datetime import datetime
from time import sleep

In [2]:

#Empezamos el scraping
#Obtener el HTML
URL_BASE = 'https://cuspide.com/100-mas-vendidos/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

#"Parsear" ese HTML
soup = BeautifulSoup(html_obtenido, "html.parser")
#sirve para sacar nombre y url
h3_todos = soup.find_all('h3')
list(h3_todos)
len(h3_todos)

#transformo el contenido de las lista en str, para usar metodos de str
for i,l in enumerate(h3_todos):
    h3_todos[i] = str(l) 
 
#Se puede sacar el precio, despues del indice 0
bdi_todos = soup.find_all('bdi')
list(bdi_todos)
len(bdi_todos)

#elinimo el primer elemento de la lista
bdi_todos.pop(0)

#transformo el contenido de las lista en str, para usar metodos de str
for i,l in enumerate(bdi_todos):
    bdi_todos[i] = str(l) 

#separa los str para sacar la informacion
nom_url=[]
for i,l in enumerate(h3_todos):
    nom_url.append(h3_todos[i][l.find('href=')+5 :])
    
#URL
#saca la url de la lista de string
url_=[]
for i,l in enumerate(nom_url):
    url_.append(nom_url[i][ :l.find('>')])

#quita la comilla doble del string del url
for i,l in enumerate(url_):    
    url_[i] = l.replace('"','')
   
#Consigo los nombres de los libros
nombre=[]
for i,l in enumerate(nom_url):
    nombre.append(nom_url[i][l.find('>')+1 : l.find('<')])

In [3]:
#Empezamos el scraping para sacar el monto en pesos y en dolares por cada url de libro
#Obtener el HTML
precio=[]
precio_dolar=[]
print('Tenga paciencia se va a demorar poco mas de 6 minutos')
print('Be patient, it will take a little more than 6 minutes.')


for i in range(0,len(url_)):
    URL_BASE = url_[i]
    pedido_obtenido = requests.get(URL_BASE)
    #para dar tiempo que carge la url
    sleep(2)
    html_obtenido = pedido_obtenido.text
    #"Parsear" ese HTML
    soup = BeautifulSoup(html_obtenido, "html.parser")

    #sirve para sacar precio en pesos
    preciopesos = soup.find(class_="price product-page-price").text
    precio.append(float(preciopesos[preciopesos.find('\n$\xa0')+3:preciopesos.find('\n$\xa0')+12].replace('.','').replace(',','.')))

    #sirve para sacar precio en dolares
    precio_dolar.append(float(soup.find('span',style="font-size: 1.3em").text.replace('.','').replace(',','.')))




Tenga paciencia se va a demorar poco mas de 6 minutos
Be patient, it will take a little more than 6 minutes.


In [ ]:
#scraping para la transformacion de dolares a euros
#Obtener el HTML
URL_BASE1 = 'https://www.xe.com/es/currencyconverter/convert/?Amount=1&From=USD&To=EUR'
pedido_obtenido1 = requests.get(URL_BASE1)
html_obtenido1 = pedido_obtenido1.text

#"Parsear" ese HTML
soup1 = BeautifulSoup(html_obtenido1, "html.parser")

#sirve para sacar nombre y url
body = soup1.find_all('body')
list(body)
body[0] = str(body[0])

#toma la parte decimal 0.00
indice = int(body[0].find('"result__BigRate-sc-1bsijpp-1 iGrAod"'))

a = body[0][indice + 38 : indice + 42]
#toma los decimaneles luego del 0.00
b = body[0][indice + 69: indice + 75]
#viene en formato latino, toca pasarlo a formato de usa
a = a.replace(',','.')

#concadena los calores y los vuelve un string
usd_eur = float(a+b)

In [ ]:
#Agregar columna de precio en dolares y euros
#toca trabajar como arreglo de numpy
precio_dolar = np.array(precio_dolar).round(2)
precio_euro = (precio_dolar*usd_eur).round(2)

#transforma los arreglos a lista
precio_dolar = list(precio_dolar)
precio_euro = list(precio_euro)

In [ ]:
#genera una lista con la fecha
now = datetime.now()
fecha=[]
for i in nombre:
    fecha.append(now.strftime("%Y-%m-%d"))

In [ ]:
df = pd.DataFrame({'Nombre':nombre,'URL':url_, 'Precio':precio, 'Precio_Dolar':precio_dolar, 'Precio_Euro':precio_euro       })

In [ ]:
df.to_csv('toplibrosv2.csv', index=False)